In [5]:
# 模块导入
import pandas as pd
import numpy as np
import yfinance as yf
import talib
import backtrader as bt

In [4]:
# ===== 猴子补丁：为 numpy 添加 bool8 属性 =====
import numpy as np
if not hasattr(np, 'bool8'):
    np.bool8 = np.bool_  # 使用 numpy 自带的 bool_ 类型

In [6]:
from backtrader_plotting import Bokeh  # 用于回测结果展示

Loading BokehJS ...

In [12]:
# ==============================
# 数据处理模块
# ==============================

def flatten_yf_columns(df: pd.DataFrame) -> pd.DataFrame:
    """
    将 yfinance 下载的数据 DataFrame 列索引进行扁平化处理，并统一为小写格式。
    适用于单只或多只股票的数据，不依赖任何特定的列名或股票名称。
    """
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [
            "_".join(tuple(filter(None, col))).lower()
            for col in df.columns.values
        ]
    else:
        df.columns = [col.lower() for col in df.columns]
    return df

def rename_columns_for_backtrader(df: pd.DataFrame, ticker: str) -> pd.DataFrame:
    """
    如果 DataFrame 列名带有股票代码前缀（例如 aapl_open），则去除前缀以满足 backtrader 的要求。
    """
    df_tmp = df.filter(regex=f'{ticker}$|_{ticker}')

    if any(col.startswith(prefix) for col in df.columns):
        df = df.rename(columns=lambda x: x[len(prefix):] if x.startswith(prefix) else x)
    return df


def fetch_data(ticker: str, start: str, end: str, interval: str = "30m") -> pd.DataFrame:
    """
    使用 yfinance 下载指定股票在一定时间范围内的数据，
    支持自定义时间颗粒度（默认为 30 分钟）。
    下载后先扁平化列索引，再将列名转换为 backtrader 需要的格式（open, high, low, close, volume）。
    """
    df = yf.download(ticker, start=start, end=end, interval=interval)
    df = flatten_yf_columns(df)
    df = rename_columns_for_backtrader(df, ticker)
    return df

In [13]:
# ==============================
# 策略模块：RSI 策略
# ==============================

class RsiStrategy(bt.Strategy):
    params = dict(
        period=14,      # RSI 计算周期
        oversold=30,    # 超卖阈值
        overbought=70   # 超买阈值
    )
    
    def __init__(self):
        self.dataclose = self.datas[0].close

    def next(self):
        if len(self.data) < self.params.period:
            return

        closes = [self.data.close[-i] for i in range(self.params.period - 1, -1, -1)]
        closes_array = np.array(closes, dtype=float)
        rsi_val = talib.RSI(closes_array, timeperiod=self.params.period)[-1]

        if not self.position and rsi_val < self.params.oversold:
            self.buy()  # 开仓买入
        elif self.position and rsi_val > self.params.overbought:
            self.close()  # 平仓


In [14]:
# ==============================
# 回测模块
# ==============================

def run_backtest(data: pd.DataFrame) -> bt.Cerebro:
    """
    初始化 Cerebro 引擎，添加数据、策略、资金、佣金和仓位控制，并运行回测。
    """
    cerebro = bt.Cerebro()
    cerebro.addstrategy(RsiStrategy)

    datafeed = bt.feeds.PandasData(dataname=data)
    cerebro.adddata(datafeed)

    cerebro.broker.setcash(100000.0)
    cerebro.addsizer(bt.sizers.FixedSize, stake=10)
    cerebro.broker.setcommission(commission=0.001)

    cerebro.run()
    return cerebro

In [15]:
# ==============================
# 结果展示模块
# ==============================

def plot_results(cerebro: bt.Cerebro):
    """
    使用 backtrader_plotting 包中的 Bokeh 后端展示回测结果。
    """
    b = Bokeh(style='bar', plot_mode='single')
    b.plot(cerebro)

In [16]:
from bokeh.io import output_notebook
output_notebook()


Loading BokehJS ...

In [19]:
ticker = 'AAPL'
# 注意：对于 30 分钟数据，时间范围通常不宜过长，否则可能无法获取完整数据
start_date = '2025-02-20'
end_date = '2025-03-04'
interval = "30m"  # 30分钟颗粒度

data = fetch_data(ticker, start_date, end_date, interval)
print("数据预览：")
print(data.head())

[*********************100%***********************]  1 of 1 completed

数据预览：
                           close_aapl   high_aapl    low_aapl   open_aapl  \
Datetime                                                                    
2025-02-20 14:30:00+00:00  244.910004  246.779999  244.610001  244.970001   
2025-02-20 15:00:00+00:00  244.744995  245.259995  244.289993  244.899994   
2025-02-20 15:30:00+00:00  245.070007  245.353897  244.589996  244.740005   
2025-02-20 16:00:00+00:00  246.130600  246.699997  245.080002  245.115005   
2025-02-20 16:30:00+00:00  245.274994  246.210007  245.095001  246.125000   

                           volume_aapl  
Datetime                                
2025-02-20 14:30:00+00:00      4760916  
2025-02-20 15:00:00+00:00      2298558  
2025-02-20 15:30:00+00:00      1604088  
2025-02-20 16:00:00+00:00      2682496  
2025-02-20 16:30:00+00:00      2351903  


In [ ]:
data.filter

In [21]:
cerebro = run_backtest(data)
print('初始资金: %.2f' % cerebro.broker.getvalue())

初始资金: nan


In [27]:
# ===== 猴子补丁：为 numpy 添加 bool8 和 object 属性 =====
import numpy as np
if not hasattr(np, 'bool8'):
    np.bool8 = np.bool_  # 使用 numpy 自带的 bool_ 类型
if not hasattr(np, 'object'):
    np.object = object  # 兼容 backtrader_plotting 的引用

# ===== 模块导入 =====
import pandas as pd
import yfinance as yf
import talib
import backtrader as bt
from backtrader_plotting import Bokeh  # 用于回测结果展示

# ==============================
# 数据处理模块
# ==============================
def flatten_yf_columns(df: pd.DataFrame) -> pd.DataFrame:
    """
    扁平化 yfinance 下载的数据的多级列索引，并转换为小写格式
    """
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [
            "_".join(tuple(filter(None, col))).lower()
            for col in df.columns.values
        ]
    else:
        df.columns = [col.lower() for col in df.columns]
    return df

def rename_columns_for_backtrader(df: pd.DataFrame, ticker: str) -> pd.DataFrame:
    """
    当列名中含有股票代码（可能在前缀或后缀，如 aapl_open 或 open_aapl）时，
    去除该部分，使列名符合 backtrader 要求（open, high, low, close, volume）
    """
    ticker_lower = ticker.lower()
    
    def rename(col_name: str) -> str:
        prefix = ticker_lower + "_"
        if col_name.startswith(prefix):
            return col_name[len(prefix):]
        suffix = "_" + ticker_lower
        if col_name.endswith(suffix):
            return col_name[:-len(suffix)]
        return col_name

    df = df.rename(columns=rename)
    return df

def fetch_data(ticker: str, start: str, end: str, interval: str = "30m") -> pd.DataFrame:
    """
    下载指定时间范围内的股票数据，并处理为 backtrader 可用的格式。
    注意：Yahoo Finance 的 30 分钟数据仅支持最近60天的数据，请确保日期范围正确。
    """
    df = yf.download(ticker, start=start, end=end, interval=interval)
    df = flatten_yf_columns(df)
    df = rename_columns_for_backtrader(df, ticker)
    return df

# ==============================
# 策略模块：RSI 策略
# ==============================
class RsiStrategy(bt.Strategy):
    params = dict(
        period=14,      # RSI 计算周期
        oversold=30,    # 超卖阈值
        overbought=70   # 超买阈值
    )
    
    def __init__(self):
        self.dataclose = self.datas[0].close

    def next(self):
        if len(self.data) < self.params.period:
            return

        closes = [self.data.close[-i] for i in range(self.params.period - 1, -1, -1)]
        closes_array = np.array(closes, dtype=float)
        rsi_val = talib.RSI(closes_array, timeperiod=self.params.period)[-1]

        if not self.position and rsi_val < self.params.oversold:
            self.buy()  # 开仓买入
        elif self.position and rsi_val > self.params.overbought:
            self.close()  # 平仓

# ==============================
# 回测模块
# ==============================
def run_backtest(data: pd.DataFrame, ticker: str):
    """
    初始化 Cerebro 引擎，添加数据、策略、资金、佣金和仓位控制，并运行回测。
    这里关键的一步是为数据源设置 _name 属性，
    以便 backtrader_plotting 的 labelizer 能正确识别数据类型。
    """
    cerebro = bt.Cerebro()
    cerebro.addstrategy(RsiStrategy)

    datafeed = bt.feeds.PandasData(dataname=data)
    datafeed._name = ticker.upper()  # 设置数据名称，解决 labelizer 报错
    cerebro.adddata(datafeed)

    cerebro.broker.setcash(100000.0)
    cerebro.addsizer(bt.sizers.FixedSize, stake=10)
    cerebro.broker.setcommission(commission=0.001)

    strategies = cerebro.run()
    return cerebro, strategies[0]

# ==============================
# 结果展示模块
# ==============================
def plot_results(strategy):
    """
    使用 backtrader_plotting 的 Bokeh 后端展示回测结果，
    传入的对象必须是策略实例，而非 Cerebro 对象。
    """
    b = Bokeh(style='bar', plot_mode='single')
    b.plot(strategy)

# ==============================
# 主程序入口
# ==============================
if __name__ == '__main__':
    ticker = 'AAPL'
    # 请确保日期范围在最近60天内，例如（假设当前日期为2025-03-05）：
    start_date = '2025-02-01'
    end_date = '2025-03-05'
    interval = "30m"  # 30分钟颗粒度

    data = fetch_data(ticker, start_date, end_date, interval)
    print("数据预览：")
    print(data.head())

    if data.empty:
        print("数据为空，请检查日期范围是否在最近60天内。")
    else:
        cerebro, strategy = run_backtest(data, ticker)
        print('初始资金: %.2f' % cerebro.broker.getvalue())
        plot_results(strategy)


/var/folders/lb/hgfyjd8d6jdgjkfcksmwvbd80000gq/T/ipykernel_39727/3918416547.py:5: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, 'object'):
[*********************100%***********************]  1 of 1 completed

数据预览：
                                close        high         low        open  \
Datetime                                                                    
2025-02-03 14:30:00+00:00  228.839996  231.830002  228.770004  229.440002   
2025-02-03 15:00:00+00:00  229.259995  229.460007  226.634995  228.809998   
2025-02-03 15:30:00+00:00  227.889999  229.819901  227.880005  229.259995   
2025-02-03 16:00:00+00:00  226.419998  228.449997  226.080002  227.923203   
2025-02-03 16:30:00+00:00  226.380005  227.130005  225.710007  226.410004   

                             volume  
Datetime                             
2025-02-03 14:30:00+00:00  13364197  
2025-02-03 15:00:00+00:00   7834052  
2025-02-03 15:30:00+00:00   4801812  
2025-02-03 16:00:00+00:00   5546159  
2025-02-03 16:30:00+00:00   4101162  
初始资金: 100000.00


In [31]:
# ===== 猴子补丁：为 numpy 添加 bool8 和 object 属性 =====
import numpy as np
if not hasattr(np, 'bool8'):
    np.bool8 = np.bool_  # 使用 numpy 自带的 bool_ 类型
if not hasattr(np, 'object'):
    np.object = object  # 兼容 backtrader_plotting 的引用

# ===== 模块导入 =====
import pandas as pd
import yfinance as yf
import talib
import backtrader as bt
from backtrader_plotting import Bokeh  # 用于回测结果展示
from bokeh.io import output_notebook  # 用于在 Notebook 中显示 Bokeh 图表
from IPython.display import IFrame, display

# 调用 output_notebook 以在 Notebook 中加载 BokehJS
output_notebook()

# ==============================
# 数据处理模块
# ==============================
def flatten_yf_columns(df: pd.DataFrame) -> pd.DataFrame:
    """
    扁平化 yfinance 下载的数据的多级列索引，并转换为小写格式
    """
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [
            "_".join(tuple(filter(None, col))).lower()
            for col in df.columns.values
        ]
    else:
        df.columns = [col.lower() for col in df.columns]
    return df

def rename_columns_for_backtrader(df: pd.DataFrame, ticker: str) -> pd.DataFrame:
    """
    当列名中含有股票代码（可能在前缀或后缀，如 aapl_open 或 open_aapl）时，
    去除该部分，使列名符合 backtrader 要求（open, high, low, close, volume）
    """
    ticker_lower = ticker.lower()
    
    def rename(col_name: str) -> str:
        prefix = ticker_lower + "_"
        if col_name.startswith(prefix):
            return col_name[len(prefix):]
        suffix = "_" + ticker_lower
        if col_name.endswith(suffix):
            return col_name[:-len(suffix)]
        return col_name

    df = df.rename(columns=rename)
    return df

def fetch_data(ticker: str, start: str, end: str, interval: str = "30m") -> pd.DataFrame:
    """
    下载指定时间范围内的股票数据，并处理为 backtrader 可用的格式。
    注意：Yahoo Finance 的 30 分钟数据仅支持最近60天的数据，请确保日期范围正确。
    """
    df = yf.download(ticker, start=start, end=end, interval=interval)
    df = flatten_yf_columns(df)
    df = rename_columns_for_backtrader(df, ticker)
    return df

# ==============================
# 策略模块：RSI 策略
# ==============================
class RsiStrategy(bt.Strategy):
    params = dict(
        period=14,      # RSI 计算周期
        oversold=30,    # 超卖阈值
        overbought=70   # 超买阈值
    )
    
    def __init__(self):
        self.dataclose = self.datas[0].close

    def next(self):
        if len(self.data) < self.params.period:
            return

        closes = [self.data.close[-i] for i in range(self.params.period - 1, -1, -1)]
        closes_array = np.array(closes, dtype=float)
        rsi_val = talib.RSI(closes_array, timeperiod=self.params.period)[-1]

        if not self.position and rsi_val < self.params.oversold:
            self.buy()  # 开仓买入
        elif self.position and rsi_val > self.params.overbought:
            self.close()  # 平仓

# ==============================
# 回测模块
# ==============================
def run_backtest(data: pd.DataFrame, ticker: str):
    """
    初始化 Cerebro，引入数据、策略、资金、佣金和仓位控制，并运行回测。
    关键步骤：为数据源设置 _name 属性，便于 backtrader_plotting 正确识别数据类型。
    """
    cerebro = bt.Cerebro()
    cerebro.addstrategy(RsiStrategy)

    datafeed = bt.feeds.PandasData(dataname=data)
    datafeed._name = ticker.upper()  # 设置数据名称，解决 labelizer 报错
    cerebro.adddata(datafeed)

    cerebro.broker.setcash(100000.0)
    cerebro.addsizer(bt.sizers.FixedSize, stake=10)
    cerebro.broker.setcommission(commission=0.001)

    strategies = cerebro.run()
    return cerebro, strategies[0]

# ==============================
# 结果展示模块
# ==============================
def plot_results(strategy):
    """
    使用 backtrader_plotting 的 Bokeh 后端生成回测图表，并将图表保存为 HTML 文件，
    然后通过 IFrame 嵌入显示。
    """
    # 这里选择 'single' 模式，也可以尝试 'multiple' 模式
    b = Bokeh(style='bar', plot_mode='single')
    pages = b.plot(strategy)
    
    # 假设我们只取第一个页面的图表
    if pages:
        filename = 'backtrader_plot.html'
        pages[0].save(filename)
        print(f'图表已保存到 {filename}')
        display(IFrame(src=filename, width=900, height=600))
    else:
        print("未生成图表，请检查策略回测数据。")

# ==============================
# 主程序入口
# ==============================
if __name__ == '__main__':
    ticker = 'AAPL'
    # 请确保日期范围在最近60天内，例如（假设当前日期为2025-03-05）：
    start_date = '2025-02-01'
    end_date = '2025-03-05'
    interval = "30m"  # 30分钟颗粒度

    data = fetch_data(ticker, start_date, end_date, interval)
    print("数据预览：")
    print(data.head())

    if data.empty:
        print("数据为空，请检查日期范围是否在最近60天内。")
    else:
        cerebro, strategy = run_backtest(data, ticker)
        print('初始资金: %.2f' % cerebro.broker.getvalue())
        plot_results(strategy)


Loading BokehJS ...

[*********************100%***********************]  1 of 1 completed

数据预览：
                                close        high         low        open  \
Datetime                                                                    
2025-02-03 14:30:00+00:00  228.839996  231.830002  228.770004  229.440002   
2025-02-03 15:00:00+00:00  229.259995  229.460007  226.634995  228.809998   
2025-02-03 15:30:00+00:00  227.889999  229.819901  227.880005  229.259995   
2025-02-03 16:00:00+00:00  226.419998  228.449997  226.080002  227.923203   
2025-02-03 16:30:00+00:00  226.380005  227.130005  225.710007  226.410004   

                             volume  
Datetime                             
2025-02-03 14:30:00+00:00  13364197  
2025-02-03 15:00:00+00:00   7834052  
2025-02-03 15:30:00+00:00   4801812  
2025-02-03 16:00:00+00:00   5546159  
2025-02-03 16:30:00+00:00   4101162  
初始资金: 100000.00


AttributeError: 'FigurePage' object has no attribute 'save'

In [33]:
import datetime
import pandas as pd
import backtrader as bt
import yfinance as yf

# --- 扁平化函数 ---
def flatten_yf_columns(df: pd.DataFrame) -> pd.DataFrame:
    """
    将 yfinance 下载的数据 DataFrame 列索引进行扁平化处理，并统一为小写格式。
    支持单只或多只股票的数据格式。
    """
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [
            "_".join(tuple(filter(None, col))).lower()
            for col in df.columns.values
        ]
    else:
        df.columns = [col.lower() for col in df.columns]
    return df

# --- 通用的列名标准化函数 ---
def standardize_columns(df: pd.DataFrame) -> pd.DataFrame:
    """
    如果除日期外的所有列都以相同后缀结尾（例如 _aapl），则去除后缀，
    保留 open、high、low、close、volume 等标准字段名称。
    """
    # 保证日期列名称为 "datetime"
    if "datetime" not in df.columns and "date" in df.columns:
        df.rename(columns={"date": "datetime"}, inplace=True)
    
    # 对非日期列，如果存在下划线，则取下划线前部分
    new_cols = {}
    for col in df.columns:
        if col != "datetime" and "_" in col:
            new_cols[col] = col.split("_")[0]
    if new_cols:
        df.rename(columns=new_cols, inplace=True)
    return df

# --- RSI 策略定义 ---
class RsiStrategy(bt.Strategy):
    params = (
        ("period", 14),      # RSI 计算周期
        ("overbought", 70),  # 超买阈值
        ("oversold", 30),    # 超卖阈值
    )
    
    def log(self, txt, dt=None):
        dt = dt or self.datas[0].datetime.date(0)
        print(f"{dt.isoformat()} {txt}")
    
    def __init__(self):
        self.dataclose = self.datas[0].close
        self.rsi = bt.indicators.RelativeStrengthIndex(self.datas[0], period=self.params.period)
    
    def next(self):
        # 当未持仓且 RSI 小于超卖阈值时，买入
        if not self.position:
            if self.rsi[0] < self.params.oversold:
                self.log(f"BUY CREATE, Price: {self.dataclose[0]:.2f}")
                self.buy()
        # 当持仓且 RSI 大于超买阈值时，卖出
        else:
            if self.rsi[0] > self.params.overbought:
                self.log(f"SELL CREATE, Price: {self.dataclose[0]:.2f}")
                self.sell()

# --- 设定日期范围（确保在最近 30 天内） ---
end_date = datetime.datetime.today()
start_date = end_date - datetime.timedelta(days=30)

print(f"Downloading data from {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")

# --- 下载 AAPL 的 5 分钟级数据 ---
df = yf.download(
    tickers="AAPL",
    start=start_date.strftime('%Y-%m-%d'),
    end=end_date.strftime('%Y-%m-%d'),
    interval="5m"
)

# --- 扁平化列名 ---
df = flatten_yf_columns(df)
if "datetime" in df.columns:
    df["datetime"] = pd.to_datetime(df["datetime"])  # 转换为 datetime 类型
    df.set_index("datetime", inplace=True)           # 将 datetime 列设为索引

print("Data head after flattening:")
print(df.head())

if df.empty:
    print("未能下载数据。请确认所请求的日期范围在最近 60 天内且 Yahoo Finance 提供 AAPL 的5分钟数据。")
else:
    # 将索引转换为普通列后，确保 datetime 列依然为 datetime 类型
    df.reset_index(inplace=True)
    df.columns = [col.lower() for col in df.columns]
    if "datetime" in df.columns:
        df["datetime"] = pd.to_datetime(df["datetime"])
    
    # 进行通用的列名标准化处理
    df = standardize_columns(df)
    print("Data head after standardizing columns:")
    print(df.head())
    
    # --- 初始化 Backtrader 并加载数据 ---
    cerebro = bt.Cerebro()
    cerebro.addstrategy(RsiStrategy)
    
    # 使用 bt.feeds.PandasData 加载数据，并显式指定 datetime 列
    data_feed = bt.feeds.PandasData(
        dataname=df,
        datetime='datetime',
        timeframe=bt.TimeFrame.Minutes,
        compression=5,
        fromdate=start_date,
        todate=end_date
    )
    # 显式设置数据名称及绘图名称，便于 backtrader_plotting 识别
    data_feed._name = 'AAPL'
    data_feed.plotinfo.plotname = 'AAPL'
    
    cerebro.adddata(data_feed)
    
    cerebro.broker.setcash(100000.0)
    
    print(f"初始资金: {cerebro.broker.getvalue():.2f}")
    
    cerebro.run()
    
    print(f"回测结束资金: {cerebro.broker.getvalue():.2f}")
    
    # --- 可视化（可选） ---
    try:
        from backtrader_plotting import Bokeh
        from backtrader_plotting.schemes import Tradimo
        b = Bokeh(style='bar', plot_mode='single', scheme=Tradimo())
        cerebro.plot(b)
    except ImportError:
        print("未安装 backtrader_plotting，使用默认 matplotlib 绘图。")
        cerebro.plot()


[*********************100%***********************]  1 of 1 completed

Data head after flattening:
                           close_aapl   high_aapl    low_aapl   open_aapl  \
Datetime                                                                    
2025-02-03 14:30:00+00:00  230.072693  231.470001  229.440002  229.440002   
2025-02-03 14:35:00+00:00  231.449997  231.830002  229.684998  230.089996   
2025-02-03 14:40:00+00:00  230.179993  231.470001  229.550095  231.425003   
2025-02-03 14:45:00+00:00  229.899994  230.234695  229.610001  230.225006   
2025-02-03 14:50:00+00:00  229.350006  230.479996  229.289993  229.770004   

                           volume_aapl  
Datetime                                
2025-02-03 14:30:00+00:00      7538679  
2025-02-03 14:35:00+00:00      1665615  
2025-02-03 14:40:00+00:00      1006650  
2025-02-03 14:45:00+00:00       873576  
2025-02-03 14:50:00+00:00      1195286  
Data head after standardizing columns:
                   datetime       close        high         low        open  \
0 2025-02-03 14:30:00+00:00

2025-02-21 BUY CREATE, Price: 246.00
2025-02-24 SELL CREATE, Price: 248.07
2025-02-25 BUY CREATE, Price: 245.93
2025-02-25 SELL CREATE, Price: 249.23
2025-02-25 BUY CREATE, Price: 246.73
2025-02-27 SELL CREATE, Price: 242.16
2025-02-27 BUY CREATE, Price: 239.20
2025-02-28 SELL CREATE, Price: 241.07
2025-03-03 BUY CREATE, Price: 239.02
回测结束资金: 100011.23


In [2]:
import datetime
import pandas as pd
import backtrader as bt
import yfinance as yf
from strategy import RsiStrategy  # 从单独模块中导入策略

# --- 扁平化函数 ---
def flatten_yf_columns(df: pd.DataFrame) -> pd.DataFrame:
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [
            "_".join(tuple(filter(None, col))).lower()
            for col in df.columns.values
        ]
    else:
        df.columns = [col.lower() for col in df.columns]
    return df

# --- 通用的列名标准化函数 ---
def standardize_columns(df: pd.DataFrame) -> pd.DataFrame:
    if "datetime" not in df.columns and "date" in df.columns:
        df.rename(columns={"date": "datetime"}, inplace=True)
    new_cols = {}
    for col in df.columns:
        if col != "datetime" and "_" in col:
            new_cols[col] = col.split("_")[0]
    if new_cols:
        df.rename(columns=new_cols, inplace=True)
    return df

# --- 设定日期范围（确保在最近 30 天内） ---
end_date = datetime.datetime.today()
start_date = end_date - datetime.timedelta(days=30)

print(f"Downloading data from {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")

# --- 下载 AAPL 的 5 分钟级数据 ---
df = yf.download(
    tickers="AAPL",
    start=start_date.strftime('%Y-%m-%d'),
    end=end_date.strftime('%Y-%m-%d'),
    interval="5m"
)

# --- 扁平化列名 ---
df = flatten_yf_columns(df)
if "datetime" in df.columns:
    df["datetime"] = pd.to_datetime(df["datetime"])
    df.set_index("datetime", inplace=True)

print("Data head after flattening:")
print(df.head())

if df.empty:
    print("未能下载数据。请确认所请求的日期范围在最近 60 天内且 Yahoo Finance 提供 AAPL 的5分钟数据。")
else:
    df.reset_index(inplace=True)
    df.columns = [col.lower() for col in df.columns]
    if "datetime" in df.columns:
        df["datetime"] = pd.to_datetime(df["datetime"])
    df = standardize_columns(df)
    print("Data head after standardizing columns:")
    print(df.head())
    
    # --- 初始化 Backtrader 并加载数据 ---
    cerebro = bt.Cerebro()
    
    # 参数优化：使用 optstrategy
    cerebro.optstrategy(
        RsiStrategy,
        period=range(10, 20, 2),       # 10, 12, 14, 16, 18
        overbought=range(65, 75, 2),   # 65, 67, 69, 71, 73
        oversold=range(25, 35, 2)      # 25, 27, 29, 31, 33
    )
    
    data_feed = bt.feeds.PandasData(
        dataname=df,
        datetime='datetime',
        timeframe=bt.TimeFrame.Minutes,
        compression=5,
        fromdate=start_date,
        todate=end_date
    )
    data_feed._name = 'AAPL'
    data_feed.plotinfo.plotname = 'AAPL'
    
    cerebro.adddata(data_feed)
    
    cerebro.broker.setcash(100000.0)
    
    print(f"初始资金: {cerebro.broker.getvalue():.2f}")
    
    # --- 运行优化 ---
    opt_results = cerebro.run()  # 参数优化返回结果为多重列表
    
    print("参数优化结果：")
    for run in opt_results:
        for strat in run:
            params = strat.params
            final_value = strat.broker.getvalue()
            print(f"Period: {params.period}, Overbought: {params.overbought}, Oversold: {params.oversold} -> Final Value: {final_value:.2f}")


YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed

Data head after flattening:
                           close_aapl   high_aapl    low_aapl   open_aapl  \
Datetime                                                                    
2025-02-03 14:30:00+00:00  230.072693  231.470001  229.440002  229.440002   
2025-02-03 14:35:00+00:00  231.449997  231.830002  229.684998  230.089996   
2025-02-03 14:40:00+00:00  230.179993  231.470001  229.550095  231.425003   
2025-02-03 14:45:00+00:00  229.899994  230.234695  229.610001  230.225006   
2025-02-03 14:50:00+00:00  229.350006  230.479996  229.289993  229.770004   

                           volume_aapl  
Datetime                                
2025-02-03 14:30:00+00:00      7538679  
2025-02-03 14:35:00+00:00      1665615  
2025-02-03 14:40:00+00:00      1006650  
2025-02-03 14:45:00+00:00       873576  
2025-02-03 14:50:00+00:00      1195286  
Data head after standardizing columns:
                   datetime       close        high         low        open  \
0 2025-02-03 14:30:00+00:00

2025-02-04 BUY CREATE, Price: 226.99
2025-02-04 SELL CREATE, Price: 229.75
2025-02-05 BUY CREATE, Price: 229.24
2025-02-05 SELL CREATE, Price: 230.70
2025-02-07 BUY CREATE, Price: 230.29
2025-02-10 SELL CREATE, Price: 229.80
2025-02-10 BUY CREATE, Price: 227.88
2025-02-11 SELL CREATE, Price: 231.10
2025-02-12 BUY CREATE, Price: 231.19
2025-02-12 SELL CREATE, Price: 233.86
2025-02-18 BUY CREATE, Price: 243.52
2025-02-18 SELL CREATE, Price: 243.70
2025-02-21 BUY CREATE, Price: 246.49
2025-02-24 SELL CREATE, Price: 247.23
2025-02-25 BUY CREATE, Price: 245.93
2025-02-25 SELL CREATE, Price: 248.06
2025-02-25 BUY CREATE, Price: 246.73
2025-02-27 SELL CREATE, Price: 241.60
2025-02-27 BUY CREATE, Price: 240.20
2025-02-28 SELL CREATE, Price: 238.55
2025-03-03 BUY CREATE, Price: 239.56
2025-03-04 SELL CREATE, Price: 238.95
2025-03-04 BUY CREATE, Price: 235.86
2025-02-04 BUY CREATE, Price: 226.99
2025-02-04 SELL CREATE, Price: 230.37
2025-02-05 BUY CREATE, Price: 229.24
2025-02-05 SELL CREATE, Pr

AttributeError: 'OptReturn' object has no attribute 'broker'